In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/gena_data/gena_data_final.csv')
df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,NEW00000,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",NEW00001,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,NEW00000,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",NEW00001,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,NEW00002,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=99b77738437fb7e6d41866439f88a156e9e1dbac62d1bd091fb40010d7248fd3
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import itertools
import os
from time import sleep
import re
from pandas.core.frame import DataFrame


In [ ]:
class Gena:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_nodes(self, node_df):
        with self.driver.session() as session:
            create_nodes_query = "CREATE "
            nodes = []
            for idx, row in node_df.iterrows():
                unique_id = row['ID']
                name = row['Preferred label'].replace('"', '')
                label = row['Label'].upper()
                synonyms = row['Synonyms'].replace('"', '')
                nodes.append("(n" + str(idx) + ":" + label + " {id: " + f'"{unique_id}"' + ", name: " + f'"{name}"' + ", synonyms:" + f'"{synonyms}"' + "})")

#         with open("test_nodes.txt", "w", encoding="utf-8") as f:
#             print("write query to file.")
#             f.write(create_nodes_query + ",".join(nodes))
            session.run(create_nodes_query + ",".join(nodes))

    def create_relationships(self, relation_df):
        with self.driver.session() as session:
            for idx, row in relation_df.iterrows():
                id_1 = row['ID_1']
                label_1 = row['Type_E1'].upper()
                id_2 = row['ID_2']
                label_2 = row['Type_E2'].upper()
                relation = re.sub('[^A-Za-z0-9|_]+', '', row['Relation'].upper().replace(' ', '_'))

                had_relation = [result for result in session.run("MATCH p = (node_1)-[r:`CID`]->(node_2) WHERE (node_1.id = $id_1 AND node_2.id = $id_2) RETURN nodes(p)", id_1=id_1, id_2=id_2)]
                if len(had_relation) > 0:
                    try:
                        pmid_source = str(int(row['PMID']))
                        create_relation = f'MATCH (node_1)-[r:CID]->(node_2)\
                        \nWHERE (node_1.id = "{id_1}" AND node_2.id = "{id_2}")\
                        \nSET r.detail = (CASE r.detail WHEN null THEN ["{relation}"] ELSE r.detail + ["{relation}"] END), r.source = r.source + ["{pmid_source}"]'

                        display(create_relation)
                        session.run(create_relation)
                    except Exception as e:
                        print(idx)
                        print(e)
                        print('---')
                else:
                    create_relation = f'MATCH (node_1), (node_2) WHERE (node_1.id = "{id_1}" AND node_2.id = "{id_2}") MERGE (node_1)-[r:`{relation}`' + ' {source:["' + row["PMID"] + '"]}]->(node_2)'
                    display(create_relation)
                    session.run(create_relation)

    def delete_data(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

In [ ]:
## Call GENA
uri = "neo4j+s://921e6f9a.databases.neo4j.io"
user = "neo4j"
password = "tQr_-EKRoD6zV5PhMgA_NqGxJCPkfobZBk140oknFk0"
gena = Gena(uri, user, password)

In [ ]:
import time
nodes_df = pd.read_csv(f"/content/drive/MyDrive/gena_data/ID_to_ENTITIES_final.csv")
pre_count = 0
count = 2000

while True:
    print(f"Create nodes from {pre_count} to {count}")
    gena.create_nodes(nodes_df[pre_count:count])
    if count > len(nodes_df):
        break
    pre_count = count
    count += 2000


Create nodes from 0 to 2000
Create nodes from 2000 to 4000
Create nodes from 4000 to 6000
Create nodes from 6000 to 8000
Create nodes from 8000 to 10000
Create nodes from 10000 to 12000
Create nodes from 12000 to 14000
Create nodes from 14000 to 16000
Create nodes from 16000 to 18000
Create nodes from 18000 to 20000
Create nodes from 20000 to 22000
Create nodes from 22000 to 24000
Create nodes from 24000 to 26000
Create nodes from 26000 to 28000
Create nodes from 28000 to 30000


In [ ]:
## Create Relationship
relationships = pd.read_csv(f"/content/drive/MyDrive/gena_data/gena_data_final.csv")
pre_count = 20000
count = 24000
reverse_relation = relationships[::-1].reset_index(drop=True)
while True:
    print(f"Create relationship from {pre_count} to {count}")
    gena.create_relationships(reverse_relation[pre_count:count])
    if count > len(reverse_relation):
        break
    pre_count = count
    count += 4000
    sleep(5)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
cid_relation = reverse_relation.iloc[[10215, 10417, 17413, 24874, 28331, 29734, 30433, 30749, 30981, 31776, 32015, 33512, 33718, 35366, 35808,
                      38663, 38967, 40255, 41356, 41922, 42688], :]
gena.create_relationships(cid_relation)

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D003024" AND node_2.id = "D020820")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["SHOW_EFFICACY_AS_TREATMENT_FOR"] ELSE r.detail + ["SHOW_EFFICACY_AS_TREATMENT_FOR"] END), r.source = r.source + ["7856623"]'

'MATCH (node_1), (node_2) WHERE (node_1.id = "C105101" AND node_2.id = "NEW17129") MERGE (node_1)-[r:`CAUSE` {source:["16436471"]}]->(node_2)'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D004837" AND node_2.id = "D006323")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["HAVE_THE_USE_TO"] ELSE r.detail + ["HAVE_THE_USE_TO"] END), r.source = r.source + ["7502643"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D007649" AND node_2.id = "D006212")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["INDUCE"] ELSE r.detail + ["INDUCE"] END), r.source = r.source + ["18022604"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D018967" AND node_2.id = "D006966")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["BE_RESERVED_GIVEN_POTENTIAL_DEVELOPMENT_OF_ATHEROSCLEROSIS_IN_OBESITY_AND_ENDOCRINE_RESPONSE_IN"] ELSE r.detail + ["BE_RESERVED_GIVEN_POTENTIAL_DEVELOPMENT_OF_ATHEROSCLEROSIS_IN_OBESITY_AND_ENDOCRINE_RESPONSE_IN"] END), r.source = r.source + ["12625994"]'

...

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D007649" AND node_2.id = "D006212")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["BE_COMPLICATED_BY"] ELSE r.detail + ["BE_COMPLICATED_BY"] END), r.source = r.source + ["27656531"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D007649" AND node_2.id = "D006212")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["CAUSE"] ELSE r.detail + ["CAUSE"] END), r.source = r.source + ["26442907"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D003024" AND node_2.id = "D009205")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["BE_EFFECTIVE_MEDICATION_FOR"] ELSE r.detail + ["BE_EFFECTIVE_MEDICATION_FOR"] END), r.source = r.source + ["35067911"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D003024" AND node_2.id = "D000380")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["CAUSE_FATAL_HEMATOPOIETIC_TOXICITY_AS"] ELSE r.detail + ["CAUSE_FATAL_HEMATOPOIETIC_TOXICITY_AS"] END), r.source = r.source + ["27368152"]'

'MATCH (node_1)-[r:CID]->(node_2)                        \nWHERE (node_1.id = "D000638" AND node_2.id = "D001919")                        \nSET r.detail = (CASE r.detail WHEN null THEN ["PRODUCE"] ELSE r.detail + ["PRODUCE"] END), r.source = r.source + ["33165142"]'